In [13]:
import pandas as pd
import altair as alt
from google.colab import files

# 1. Cargar el archivo limpio
df = pd.read_csv('/content/oscars_limpio_1997_2025-5.csv')

# 2. Palabras clave de plataformas de streaming
streaming_keywords = ['netflix', 'amazon', 'prime video', 'apple', 'hulu', 'disney+']

# 3. Función para detectar si el distribuidor es de streaming
def is_streaming(row):
    distribuidor = str(row['distribuidor']).lower()
    return any(keyword in distribuidor for keyword in streaming_keywords)

# 4. Aplicar la función
df['is_streaming'] = df.apply(is_streaming, axis=1)

# 5. Agrupar y resumir datos
summary = df.groupby(['Year', 'is_streaming'])['winner'].agg(['count', 'sum']).reset_index()
summary = summary.rename(columns={'count': 'nominations', 'sum': 'wins'})

# 6. Transformar para graficar
summary_long = summary.melt(
    id_vars=['Year', 'is_streaming'],
    value_vars=['nominations', 'wins'],
    var_name='Metric',
    value_name='Count'
)

summary_long['Distribuidor'] = summary_long['is_streaming'].map({True: 'Streaming', False: 'Tradicional'})
summary_long['Métrica'] = summary_long['Metric'].map({'nominations': 'Nominaciones', 'wins': 'Premios Ganados'})

# 7. Definir escala de color
color_scale = alt.Scale(domain=['Streaming', 'Tradicional'], range=['#e63946', '#457b9d'])

# 8. Crear gráfico
base = alt.Chart(summary_long).mark_line(point=alt.OverlayMarkDef(filled=True, size=60)).encode(
    x=alt.X('Year:O', title='Año'),
    y=alt.Y('Count:Q', title='Cantidad', axis=alt.Axis(labelFontSize=12, titleFontSize=14)),
    color=alt.Color('Distribuidor:N', title='Tipo de Distribuidor', scale=color_scale),
    strokeDash=alt.StrokeDash('Distribuidor:N', legend=None),
    tooltip=['Year', 'Distribuidor', 'Métrica', 'Count']
).properties(
    width=320,
    height=260
)

chart = base.facet(
    column=alt.Column('Métrica:N', title=None, header=alt.Header(labelFontSize=14, labelFontWeight='bold'))
).properties(
    title={
        "text": ["Evolución de la presencia de plataformas de streaming en los Premios Oscar"],
        "subtitle": ["Comparación de nominaciones y premios ganados frente a estudios tradicionales (1997–2025)"],
        "fontSize": 18,
        "subtitleFontSize": 14,
        "anchor": "start"
    }
).configure(
    background='#ffffff'
).configure_view(
    stroke=None
).configure_axis(
    labelFontSize=11,
    titleFontSize=13
).configure_legend(
    labelFontSize=12,
    titleFontSize=13
)

# 9. Guardar el gráfico como HTML
chart.save('streaming_vs_tradicional.html')
files.download('streaming_vs_tradicional.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
chart.save('streaming_vs_tradicional.html')
from google.colab import files
files.download('streaming_vs_tradicional.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>